<a href="https://colab.research.google.com/github/gizelleguerra/genAI_demos/blob/main/RAG_Llama2_Pinecone_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Llama 2+ Pinecone + LangChain**

##**Step 1: Install All the Required Pakages**

In [1]:
!pip install langchain
!pip install pypdf
!pip install unstructured
!pip install sentence_transformers
!pip install pinecone-client
!pip install llama-cpp-python
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.3/270.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py

In [3]:
!pip -q install bitsandbytes accelerate xformers einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.7 MB/s eta 0:00:00


In [2]:
!pip -q install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.2 MB/s eta 0:00:00


#**Step 2: Import All the Required Libraries**

In [5]:
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer
from langchain.chains.question_answering import load_qa_chain
import pinecone
import os, sys

In [6]:
from huggingface_hub import notebook_login
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

In [7]:
from langchain import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

#**Step 3: Load the Data**

In [8]:
# grab TIVDAK drug prescribing info from web
!mkdir -p DATA/TIVDAK;curl https://docs.seagen.com/Tivdak_Full_Ltr_Master.pdf -o DATA/TIVDAK/tivdak.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  452k  100  452k    0     0   442k      0  0:00:01  0:00:01 --:--:--  442k


In [9]:
# save here
filepath = '/content/DATA/TIVDAK/tivdak.pdf'
loader = PyPDFLoader(filepath)
document=loader.load()

In [ ]:
#!gdown "https://drive.google.com/uc?id=15hUEJQViQDxu_fnJeO_Og1hGqykCmJut&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=15hUEJQViQDxu_fnJeO_Og1hGqykCmJut&confirm=t
To: /content/The-Field-Guide-to-Data-Science.pdf
100% 30.3M/30.3M [00:00<00:00, 34.8MB/s]


In [ ]:
#loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")
#loader = PyPDFLoader("/content/yolov7paper.pdf")
#loader = PyPDFLoader("/content/The-Field-Guide-to-Data-Science.pdf")

In [ ]:
#data = loader.load()

In [10]:
len(document)

22

#**Step 4: Split the Text into Chunks**

In [ ]:
#text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

In [11]:
document_splitter=CharacterTextSplitter(separator='\n', chunk_size=1000, chunk_overlap=100)

In [13]:
docs=document_splitter.split_documents(document)

In [14]:
len(docs)

71

In [15]:
docs[0]

Document(page_content='HIGHLIGHTS OF PRESCRIBING INFORMATION   \nThese highlights do not include all the information needed to use \nTIVDAK safely and effectively. See full prescribing information for \nTIVDAK.  \nTIVDAK® (tisotumab vedotin -tftv) for injection, for intravenous use  \nInitial U.S. Approval: 2021  \n \nWARNING: OCULAR TOXICITY  \nSee full prescribing information for complete boxed warning.  \n \n• TIVDAK caused changes in the corneal epithelium and \nconjunctiva resulting in changes in vision, including severe vision \nloss, and corneal ulceration. ( 5.1) \n• Conduct an ophthalmic exam at baseline, prior to each dose, and \nas clinically indicated. ( 2.2, 5.1) \n• Adhere to premedication and required eye care before, during, \nand after infusion. ( 2.2) \n• Withhold TIVDAK until improvement and resume, reduce the \ndose, or permanently discontinue, based on severity. ( 2.3, 5.1) \n --------------------------   RECENT MAJOR CHANGES  --------------------------', metadata=

#**Step 6: Downlaod the Embeddings**

In [17]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

#**Step 7: Initializing the Pinecone**

In [32]:
# initialize pinecone
pinecone.init(
    api_key='34357661-cd66-4c00-8142-c49fc49a1b79',  # find at app.pinecone.io
    environment='us-west1-gcp-free'  # next to api key in console
)

index_name = "langchain-llama-rag" # put in the name of your pinecone index here

In [33]:
if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='cosine',
        dimension=1536,  # 1536 dim of text-embedding-ada-002
    )

In [ ]:
# PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '34357661-cd66-4c00-8142-c49fc49a1b79')
# PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'us-west1-gcp-free')

# pinecone.init(
#     api_key=PINECONE_API_KEY,  # find at app.pinecone.io
#     environment=PINECONE_API_ENV  # next to api key in console
# )

#**Step 8: Create Embeddings for Each of the Text Chunk**

In [35]:
docsearch=Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)

ForbiddenException: ignored

# If you already have an index, you can load it like this


In [24]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)


#**Step 9: Similarity Search**

In [26]:
#query="What are examples of good data science teams?"
query="what is tivdak used for?"

In [27]:
docs=docsearch.similarity_search(query)

PineconeProtocolError: ignored

In [ ]:
docs

[Document(page_content='From the results we see that if compared with YOLOv4,\nYOLOv7 has 75% less parameters, 36% less computation,\nand brings 1.5% higher AP. If compared with state-of-the-\nart YOLOR-CSP, YOLOv7 has 43% fewer parameters, 15%\nless computation, and 0.4% higher AP. In the performance\nof tiny model, compared with YOLOv4-tiny-31, YOLOv7-\ntiny reduces the number of parameters by 39% and the\namount of computation by 49%, but maintains the same AP.\nOn the cloud GPU model, our model can still have a higher', metadata={}),
 Document(page_content='If we compare YOLOv7 with YOLOR using the input\nresolution 1280, the inference speed of YOLOv7-W6 is 8\nfps faster than that of YOLOR-P6, and the detection rate is\nalso increased by 1% AP. As for the comparison between\nYOLOv7-E6 and YOLOv5-X6 (r6.1), the former has 0.9%\nAP gain than the latter, 45% less parameters and 63% less\ncomputation, and the inference speed is increased by 47%.\nYOLOv7-D6 has close inference speed to 

#**Step 9: Query the Docs to get the Answer Back (Llama 2 Model)**

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.3 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached setuptools-68.0.0-py3-none-any.whl (804 kB)
    Using cached scikit_build-0.17.6-py3-none-any.whl (84 kB)
    Using cached cmake-3.27.0-py2.py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (26.0 MB)
    Using cached ninja-1.11.1-py2.py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (145 kB)
    Using cached distro-1.8.0-py3-none-any.whl (20 kB)
    Using cached packaging-23.1-py3-none-any.whl (48 kB)
    Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
    Using cached wheel-0.41.0-py3-none-any.whl (64 kB)
    Creating /tmp/pip-build-env-nvbt437i/overlay/local/bin
    changing mode of /tmp/pip-build-env-nvbt437i/overlay/local/bin/ninja to 755
    changing m

#Import All the Required Libraries

In [ ]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from huggingface_hub import hf_hub_download
from langchain.chains.question_answering import load_qa_chain

In [ ]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

#  Quantized Models from the Hugging Face Community

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.

There are several variations available, but the ones that interest us are based on the GGLM library.

We can see the different variations that Llama-2-13B-GGML has [here](https://huggingface.co/models?search=llama%202%20ggml).



In this case, we will use the model called [Llama-2-13B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML).

 Quantization reduces precision to optimize resource usage.

Quantization is a technique to reduce the computational and memory costs of running inference by representing the weights and activations with low-precision data types like 8-bit integer ( int8 ) instead of the usual 32-bit floating point ( float32 ).

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [ ]:
n_gpu_layers = 40  # Change this value based on your model and your GPU VRAM pool.
n_batch = 256  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Loading model,
llm = LlamaCpp(
    model_path=model_path,
    max_tokens=256,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    n_ctx=1024,
    verbose=False,
)

In [ ]:
chain=load_qa_chain(llm, chain_type="stuff")

In [ ]:
query="YOLOv7 outperforms which models"
docs=docsearch.similarity_search(query)

In [ ]:
docs

[Document(page_content='From the results we see that if compared with YOLOv4,\nYOLOv7 has 75% less parameters, 36% less computation,\nand brings 1.5% higher AP. If compared with state-of-the-\nart YOLOR-CSP, YOLOv7 has 43% fewer parameters, 15%\nless computation, and 0.4% higher AP. In the performance\nof tiny model, compared with YOLOv4-tiny-31, YOLOv7-\ntiny reduces the number of parameters by 39% and the\namount of computation by 49%, but maintains the same AP.\nOn the cloud GPU model, our model can still have a higher', metadata={}),
 Document(page_content='If we compare YOLOv7 with YOLOR using the input\nresolution 1280, the inference speed of YOLOv7-W6 is 8\nfps faster than that of YOLOR-P6, and the detection rate is\nalso increased by 1% AP. As for the comparison between\nYOLOv7-E6 and YOLOv5-X6 (r6.1), the former has 0.9%\nAP gain than the latter, 45% less parameters and 63% less\ncomputation, and the inference speed is increased by 47%.\nYOLOv7-D6 has close inference speed to 

In [ ]:
chain.run(input_documents=docs, question=query)

 YOLOv7 outperforms ConvNeXt-L, Swin-L, and ConvNeXt-XL in terms of both speed and accuracy.

' YOLOv7 outperforms ConvNeXt-L, Swin-L, and ConvNeXt-XL in terms of both speed and accuracy.'

In [ ]:
query="YOLOv7 trained on which dataset"
docs=docsearch.similarity_search(query)

In [ ]:
chain.run(input_documents=docs, question=query)

 MS COCO dataset

' MS COCO dataset'

#**Step 10: Query the Docs to get the Answer Back (Hugging Face Model)**

In [ ]:
from langchain.llms import HuggingFaceHub

In [ ]:
llm=HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})

In [ ]:
chain=load_qa_chain(llm, chain_type="stuff")

In [ ]:
query="What are examples of good data science teams?"
docs=docsearch.similarity_search(query)

In [ ]:
chain.run(input_documents=docs, question=query)

'Data Science teams need a broad view of the organization. Leaders must be key advocates who meet'